# BIGSI Test 1

This is a test of the software `bigsi` <https://bigsi.readme.io/>, specifically by following the tutorial <https://bigsi.readme.io/docs/your-first-bigsi>.

# Step 1

Step 1 involves downloading the fastq files and counting kmers. This makes use of the software `fasterq-dump` to download fastq files and `jellyfish` to count kmers. The genomes to download are stored in the file `genomes.txt`.

In [1]:
NOTEBOOK_DIR=`git rev-parse --show-toplevel`

cd $NOTEBOOK_DIR
rm -rf kmers
mkdir kmers
pushd kmers

~/workspace/bigsi-examples/kmers ~/workspace/bigsi-examples


In [6]:
kmer_size=4
jellyfish_hash_size=10M
jellyfish_threads=8

for accession in `cat ../genomes.txt`;
do
    echo "Working on $accession"
    
    fastq_file=${accession}.fastq
    jellyfish_file=${accession}_${kmer_size}mers.jf
    kmer_file=${accession}_${kmer_size}mers.txt
    
    set -x
    fasterq-dump -s -o ${fastq_file} "$accession"
    jellyfish count -m ${kmer_size} -s ${jellyfish_hash_size} -t ${jellyfish_threads} --canonical -o ${jellyfish_file} ${fastq_file}
    jellyfish dump -c -t ${jellyfish_file} | cut -f 1 > ${kmer_file}
    set +x
done

echo "Done"

Working on SRR9842706
+ fasterq-dump -s -o SRR9842706.fastq SRR9842706
2019-10-01T20:39:43 fasterq-dump.2.10.0 err: fasterq-dump.c fastdump_csra() checking ouput-file 'SRR9842706.fastq' -> RC(rcExe,rcFile,rcPacking,rcName,rcExists)
+ jellyfish count -m 4 -s 10M -t 8 --canonical -o SRR9842706_4mers.jf SRR9842706.fastq
+ cut -f 1
+ jellyfish dump -c -t SRR9842706_4mers.jf
+ set +x
Done


Awesome. We've now got files named like `SRR9842706_kmers.txt` in our directory containing the canonical kmers. These should look like:

In [8]:
head -n 4 SRR9842706_4mers.txt

AAAA
AAAC
AAAG
AAAT


Let's start using these to build the `bigsi` index.

# Step 2

In step 2, we'll look at building the `bigsi` indexes for each of these kmer counts.